In [1]:

%load_ext autoreload
%autoreload 2

In [14]:
from sqlalchemy import select
from sqlalchemy.ext.asyncio import async_sessionmaker
from sqlalchemy.orm import selectinload

from farmbase.auth.models import FarmbaseUserOrganization
from farmbase.contact.models import Contact
from farmbase.database.core import engine
from farmbase.farm.models import FarmContact, Farm

schema_engine = engine.execution_options(
    schema_translate_map={
        None: "farmbase_organization_default",
    }
)

# session = sessionmaker(bind=engine)

async_session_factory = async_sessionmaker(
    bind=schema_engine,
    expire_on_commit=False,
)

FarmbaseUserOrganization.farmbase_user_id
FarmContact.id

async with async_session_factory.begin() as db_session:

    stmt = (
        select(Contact, Farm)
        .join(Contact.farm_associations)
        .join(FarmContact.farm)
        .where(Farm.location.is_not(None))
        .options(
            selectinload(Contact.farm_associations).selectinload(FarmContact.farm)
        )
        .distinct()
    )

    result = await db_session.execute(stmt)

for contact, farm in result.all():
    print(contact.name, farm.farm_name)

2025-05-28 16:11:34,182 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-28 16:11:34,183 INFO sqlalchemy.engine.Engine SELECT DISTINCT farmbase_organization_default.contact.id, farmbase_organization_default.contact.name, farmbase_organization_default.contact.phone_number, farmbase_organization_default.contact.preferred_form_of_address, farmbase_organization_default.contact.gender, farmbase_organization_default.contact.date_of_birth, farmbase_organization_default.contact.estimated_age, farmbase_organization_default.contact.role, farmbase_organization_default.contact.experience, farmbase_organization_default.contact.email, farmbase_organization_default.contact.organization_id, farmbase_organization_default.contact.created_at, farmbase_organization_default.contact.updated_at, farmbase_organization_default.farm.id AS id_1, farmbase_organization_default.farm.farm_name, ST_AsEWKB(farmbase_organization_default.farm.location) AS location, farmbase_organization_default.farm.created_at AS 